In [32]:
import pandas as pd
import numpy as np 

In [80]:
region = pd.read_csv("RAW/region.csv")
region

,region_id,region_name
0,1,Africa
1,2,America
2,3,Asia
3,4,Europe
4,5,Oceania


In [91]:
customer_nodes = pd.read_csv("RAW/Customer_Nodes.csv")

customer_nodes = customer_nodes.merge(region, 'left', 'region_id')

customer_nodes

,customer_id,region_id,node_id,start_date,end_date,region_name
0,1,3,4,2020-01-02,2020-01-03,Asia
1,2,3,5,2020-01-03,2020-01-17,Asia
2,3,5,4,2020-01-27,2020-02-18,Oceania
3,4,5,4,2020-01-07,2020-01-19,Oceania
4,5,3,3,2020-01-15,2020-01-23,Asia
...,...,...,...,...,...,...
3495,496,3,4,2020-02-25,9999-12-31,Asia
3496,497,5,4,2020-05-27,9999-12-31,Oceania
3497,498,1,2,2020-04-05,9999-12-31,Africa
3498,499,5,1,2020-02-03,9999-12-31,Oceania


In [35]:
customer_transactions = pd.read_csv("RAW/Customer_Transactions.csv")
customer_transactions

,customer_id,txn_date,txn_type,txn_amount
0,429,2020-01-21,deposit,82
1,155,2020-01-10,deposit,712
2,398,2020-01-01,deposit,196
3,255,2020-01-14,deposit,563
4,185,2020-01-29,deposit,626
...,...,...,...,...
5863,189,2020-02-03,withdrawal,870
5864,189,2020-03-22,purchase,718
5865,189,2020-02-06,purchase,393
5866,189,2020-01-22,deposit,302


A. Customer Nodes Exploration
How many unique nodes are there on the Data Bank system?
What is the number of nodes per region?
How many customers are allocated to each region?
How many days on average are customers reallocated to a different node?
What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [64]:
#How many unique nodes are there on the Data Bank system?
print("there are",customer_nodes.node_id.count(),"entries of nodes in the Data bank System")

print("these are all variations of the following ",len(customer_nodes.node_id.unique()),":",customer_nodes.node_id.unique())

print("in the following proportions", customer_nodes.groupby('node_id').size())

there are 3500 entries of nodes in the Data bank System
these are all variations of the following  5 : [4 5 3 1 2]
in the following proportions node_id
1    728
2    662
3    699
4    704
5    707
dtype: int64


In [93]:
#What is the number of nodes per region?
print("The Distribution for nodes per region is ", customer_nodes.groupby('region_name').size())

The Distribution for nodes per region is  region_name
Africa     770
America    735
Asia       714
Europe     665
Oceania    616
dtype: int64


In [98]:
#How many customers are allocated to each region?

print("The Distribution for customer per region is ",customer_nodes.groupby('region_name'))

The Distribution for customer per region is  <pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000235CA169250>


In [62]:
help(pd.Series.sort_values)

Help on function sort_values in module pandas.core.series:

sort_values(self, axis=0, ascending: Union[bool, int, Sequence[Union[bool, int]]] = True, inplace: bool = False, kind: str = 'quicksort', na_position: str = 'last', ignore_index: bool = False, key: Optional[Callable[[ForwardRef('Series')], Union[ForwardRef('Series'), ~AnyArrayLike]]] = None)
    Sort by the values.
    
    Sort a Series in ascending or descending order by some
    criterion.
    
    Parameters
    ----------
    axis : {0 or 'index'}, default 0
        Axis to direct sorting. The value 'index' is accepted for
        compatibility with DataFrame.sort_values.
    ascending : bool or list of bools, default True
        If True, sort values in ascending order, otherwise descending.
    inplace : bool, default False
        If True, perform operation in-place.
    kind : {'quicksort', 'mergesort' or 'heapsort'}, default 'quicksort'
        Choice of sorting algorithm. See also :func:`numpy.sort` for more
       